In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
from scipy import stats
import sklearn as sk

In [52]:
batting2015_df = pd.read_csv("batting2015.csv",index_col = 0)
batting2015_df = batting2015_df.convert_objects(convert_numeric=True)

pinfo_df = pd.read_csv("player_info_table.csv",index_col=0)
del pinfo_df["Team"]
pinfo_df = pinfo_df.convert_objects(convert_numeric=True)
pinfo_df.head()

batting2015_df = pd.merge(batting2015_df,pinfo_df,on="選手名")
# batting2015_df.drop("Team_x")
# batting2015_df[merged_df["守備"]=="投手"]
batting2015_df.dtypes

Team       object
背番号         int64
選手名        object
打率        float64
試合        float64
打席数       float64
打数        float64
安打        float64
本塁打       float64
打点        float64
盗塁        float64
四球        float64
死球        float64
三振        float64
犠打        float64
併殺打       float64
長打率       float64
出塁率       float64
OPS       float64
RC27      float64
XR27      float64
No.         int64
守備         object
生年月日       object
年齢          int64
年数          int64
身長          int64
体重          int64
血液型        object
出身地        object
年俸(推定)    float64
投          object
打          object
dtype: object

In [54]:
batting2015_df = batting2015_df.dropna() #NaNを消す
at_bat = batting2015_df["打数"]
hit = batting2015_df["安打"]
Iscarp = batting2015_df["Team"]=="広"
Isbat_right = batting2015_df["打"]=="右"
plt.plot(at_bat[-Iscarp],hit[-Iscarp],"o",label="NotCarp")
plt.plot(at_bat[Iscarp],hit[Iscarp],"o",label="Carp")
plt.plot(np.linspace(0,np.max(at_bat),100),np.linspace(0,np.max(at_bat),100)*.3,"--")
plt.legend(bbox_to_anchor=(.05, 1), loc=2, borderaxespad=0.)
plt.show()

In [60]:
IsoD = batting2015_df["出塁率"]-batting2015_df["打率"]
IsoP = batting2015_df["長打率"]-batting2015_df["打率"]
Isover50 = batting2015_df["打席数"]>=50
plt.subplot(1,3,1)
plt.plot(IsoD[Isover50&Isbat_right],IsoP[Isover50&Isbat_right],"o",label="Bat at R")
plt.plot(IsoD[Isover50&-Isbat_right],IsoP[Isover50&-Isbat_right],"o",label="Bat at L")
# plt.plot(IsoD[-Isover50],IsoP[-Isover50],"o")
plt.legend()
plt.xlabel("IsoD")
plt.ylabel("IsoP")

plt.subplot(1,3,2)
plt.plot(batting2015_df["XR27"][Isover50&Isbat_right],IsoP[Isover50&Isbat_right],"o",label="Bat at R")
plt.plot(batting2015_df["XR27"][Isover50&-Isbat_right],IsoP[Isover50&-Isbat_right],"o",label="Bat at L")
plt.subplot(1,3,3)
plt.plot(batting2015_df["体重"][Isover50&Isbat_right],IsoP[Isover50&Isbat_right],"o",label="Bat at R")
plt.plot(batting2015_df["体重"][Isover50&-Isbat_right],IsoP[Isover50&-Isbat_right],"o",label="Bat at L")
plt.show()

In [56]:
batting2015_df["打"][Isbat_right] = "right"
batting2015_df["打"][-Isbat_right] = "left"

C:\Users\det A\Anaconda\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [63]:
d = {
    "Bat":batting2015_df["打"],
#     "at_bat":batting2015_df["打席数"],
    "hit":batting2015_df["安打"],
    "hr":batting2015_df["本塁打"],
#     "bb":batting2015_df["四球"],
#     "deadball":batting2015_df["死球"], 
    "Price":batting2015_df["年俸(推定)"],
    "IsoD":batting2015_df["出塁率"]-batting2015_df["打率"],
    "IsoP":batting2015_df["長打率"]-batting2015_df["打率"],
    "XR27":batting2015_df["XR27"],
    "Weight": batting2015_df["体重"],
    "Height": batting2015_df["身長"]
}
df = pd.DataFrame(data=d)
sns.pairplot(df,hue="Bat")
# plt.show()
plt.savefig("bat_at_left_vs_right.png")

In [66]:
corr = df.corr()
sns.heatmap(corr)
plt.show()

In [35]:
batting2015_df["IsoP"]=IsoP
batting2015_df["IsoD"]=IsoD
batting2015_df[Isover50].sort(columns="IsoP",ascending=False).head(10)

,Team,背番号,選手名,打率,試合,打席数,打数,安打,本塁打,打点,...,年数,身長,体重,血液型,出身地,年俸(推定),投,打,IsoP,IsoD
179,ヤ,23,山田 哲人,0.329,143,646,557,183,38,100,...,5,180,76,O,兵庫,8000,右,右,0.281,0.087
5,広,55,エルドレッド,0.230,78,297,261,60,19,54,...,4,196,122,不明,アメリカ,10900,右,右,0.241,0.080
165,横,55,後藤 武敏 Ｇ．,0.206,70,76,68,14,4,11,...,13,176,84,A,静岡,2200,右,右,0.235,0.083
147,横,25,筒香 嘉智,0.317,138,568,496,157,24,93,...,6,185,97,A,和歌山,4600,右,左,0.205,0.083
148,横,2,ロペス,0.291,140,565,516,150,25,73,...,3,183,103,不明,ベネズエラ,15000,右,右,0.205,0.056
52,巨,91,堂上 剛裕,0.276,59,104,98,27,3,13,...,12,183,85,O,愛知,1500,右,左,0.204,0.025
182,ヤ,33,畠山 和洋,0.268,137,584,512,137,26,105,...,15,180,96,O,岩手,9000,右,右,0.203,0.076
162,横,26,井手 正太郎,0.255,40,102,98,25,4,13,...,14,179,85,O,宮崎,1150,右,右,0.194,0.020
51,巨,23,相川 亮二,0.313,40,114,99,31,4,17,...,21,182,86,A,千葉,6000,右,右,0.192,0.071
13,広,13,グスマン,0.230,34,116,100,23,3,12,...,1,185,91,A,ベネズエラ,10800,右,右,0.190,0.106


In [5]:
d = {
    "IsoD":IsoD,
    "IsoP":IsoP,
    "at_bat":at_bat,
    "hit":hit,
    "bb":batting2015_df["四球"],
    "deadball":batting2015_df["死球"], 
    "XR27":batting2015_df["XR27"],
    "hr":batting2015_df["本塁打"]
}
my_df = pd.DataFrame(data=d)
pd.tools.plotting.scatter_matrix(my_df,diagonal="kde")
plt.show()

In [6]:
my_df.plot(kind="scatter",x="IsoD",y="IsoP",s=my_df["at_bat"])
plt.show()

In [7]:
from sklearn.cluster import KMeans
kmeans_model = KMeans(n_clusters = 3,init="k-means++",random_state=10)
kmeans_model.fit(my_df[["IsoD","IsoP"]])

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=3, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=10, tol=0.0001,
    verbose=0)

In [8]:
labels = kmeans_model.labels_

In [9]:
for i in range(max(labels)+1):
    plt.plot(my_df["IsoD"][labels==i],my_df["IsoP"][labels==i],"o")
plt.show()

In [ ]:
kmeans_model.cluster_centers_

In [ ]:
pinfo_df = pd.read_csv("player_info_table.csv",index_col=0)
pinfo_df.head()